# Sentiment analysis

##  Set up

In [1]:
%pip install numpy pandas nltk sklearn

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np 
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string


## Load dataset

In [3]:
df = pd.read_csv(r'../resources/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

## Tiền xử lý dữ liệu

### Làm sạch dữ liệu 
- Loại bỏ các thẻ HTML
- Loại bỏ khoảng trắng thừa và dấu câu 
- Loại bỏ stop words
- Chuyển đổi chữ hoa thành chữ thường 
- Chuyển đổi nhãn sang số

#### Loại bỏ thẻ HTML

In [5]:
df['review'] = df['review'].str.replace("<br />", "")
df['review'][2]

'I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.This was the most I\'d laughed at one of Woody\'s comedies in years (dare I say a decade?). While I\'ve never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.'

#### Loại bỏ khoảng trắng thừa và dấu câu

In [6]:
df['review'] = df['review'].str.translate(str.maketrans('', '', string.punctuation))

In [7]:
df['review'][2]

'I thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air conditioned theater and watching a lighthearted comedy The plot is simplistic but the dialogue is witty and the characters are likable even the well bread suspected serial killer While some may be disappointed when they realize this is not Match Point 2 Risk Addiction I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to loveThis was the most Id laughed at one of Woodys comedies in years dare I say a decade While Ive never been impressed with Scarlet Johanson in this she managed to tone down her sexy image and jumped right into a average but spirited young womanThis may not be the crown jewel of his career but it was wittier than Devil Wears Prada and more interesting than Superman a great comedy to go see with friends'

In [8]:
df['review'] = df['review'].str.strip()
df['review'] = df['review'].str.replace(r'\s+',' ', regex=True)
df['review'][2]

'I thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air conditioned theater and watching a lighthearted comedy The plot is simplistic but the dialogue is witty and the characters are likable even the well bread suspected serial killer While some may be disappointed when they realize this is not Match Point 2 Risk Addiction I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to loveThis was the most Id laughed at one of Woodys comedies in years dare I say a decade While Ive never been impressed with Scarlet Johanson in this she managed to tone down her sexy image and jumped right into a average but spirited young womanThis may not be the crown jewel of his career but it was wittier than Devil Wears Prada and more interesting than Superman a great comedy to go see with friends'

#### Loại bỏ stop words

In [9]:
# Tải stop words từ nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [11]:
def remove_stop_words(text):
    words = text.split()
    filter_words = [word for word in words if word not in stop_words]
    return ' '.join(filter_words)

In [12]:
df['review'] = df['review'].apply(remove_stop_words)
df['review'][2]

'I thought wonderful way spend time hot summer weekend sitting air conditioned theater watching lighthearted comedy The plot simplistic dialogue witty characters likable even well bread suspected serial killer While may disappointed realize Match Point 2 Risk Addiction I thought proof Woody Allen still fully control style many us grown loveThis Id laughed one Woodys comedies years dare I say decade While Ive never impressed Scarlet Johanson managed tone sexy image jumped right average spirited young womanThis may crown jewel career wittier Devil Wears Prada interesting Superman great comedy go see friends'

#### Chuyển chữ hoa thành chữ thường 

In [13]:
df['review'] = df['review'].str.lower()
df['review'][2]

'i thought wonderful way spend time hot summer weekend sitting air conditioned theater watching lighthearted comedy the plot simplistic dialogue witty characters likable even well bread suspected serial killer while may disappointed realize match point 2 risk addiction i thought proof woody allen still fully control style many us grown lovethis id laughed one woodys comedies years dare i say decade while ive never impressed scarlet johanson managed tone sexy image jumped right average spirited young womanthis may crown jewel career wittier devil wears prada interesting superman great comedy go see friends'

#### Chuyển đổi nhãn sang số

In [14]:
def sentiment_encode(text):
    if(text == "positive"):
        return 1
    return -1

In [15]:
df['sentiment'] = df['sentiment'].apply(sentiment_encode)

### Chia tập dữ liệu

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


In [19]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train1, X_test1, y_train1, y_test1 = train_test_split(df['review'], df['sentiment'], test_size=0.3, random_state=42)

In [20]:
# Chuyển đổi văn bản thành các đặc trưng số sử dụng TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=10000,)
X_train_tfidf = vectorizer.fit_transform(X_train1)
X_test_tfidf = vectorizer.transform(X_test1)

In [21]:
X_train_tfidf[0]

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 51 stored elements and shape (1, 10000)>

In [24]:
sentiment = y_train1.unique()


## Mô hình SVM

### Dùng siêu tham số

In [25]:
from sklearn.decomposition import TruncatedSVD
# 5. Giảm chiều với Truncated SVD
svd = TruncatedSVD(n_components=2500, random_state=42)
X_train_reduced = svd.fit_transform(X_train_tfidf)
X_test_reduced = svd.transform(X_test_tfidf)

In [26]:
from sklearn.svm import SVC
svm_classifier = SVC(
    C=50, 
    coef0=0.5, 
    gamma=0.01, 
    kernel='poly', 
    random_state=42
)

In [27]:
svm_classifier.fit(X_train_reduced,y_train1)

SVC(C=50, coef0=0.5, gamma=0.01, kernel='poly', random_state=42)

In [29]:
from sklearn.metrics import accuracy_score, classification_report

In [30]:
# Dự đoán trên tập kiểm tra
y_pred_train1 = svm_classifier.predict(X_train_reduced)

# Đánh giá mô hình
accuracy_train1 = accuracy_score(y_train1, y_pred_train1)
report_train1 = classification_report(y_train1, y_pred_train1)

print(f'Accuracy: {accuracy_train1}')
print('Classification Report:')
print(report_train1)

Accuracy: 0.9048857142857143
Classification Report:
              precision    recall  f1-score   support

          -1       0.91      0.89      0.90     17589
           1       0.90      0.92      0.91     17411

    accuracy                           0.90     35000
   macro avg       0.91      0.90      0.90     35000
weighted avg       0.91      0.90      0.90     35000



In [31]:
# Dự đoán trên tập kiểm tra
y_pred1 = svm_classifier.predict(X_test_reduced)

# Đánh giá mô hình
accuracy1 = accuracy_score(y_test1, y_pred1)
report1 = classification_report(y_test1, y_pred1)

print(f'Accuracy: {accuracy1}')
print('Classification Report:')
print(report1)

Accuracy: 0.8916666666666667
Classification Report:
              precision    recall  f1-score   support

          -1       0.90      0.88      0.89      7411
           1       0.88      0.91      0.89      7589

    accuracy                           0.89     15000
   macro avg       0.89      0.89      0.89     15000
weighted avg       0.89      0.89      0.89     15000



### Word2Vec

In [1]:
import numpy as np
import pickle
with open("../resources/X_w2v_train.pkl", "rb") as f:
    X_train = pickle.load(f)
with open("../resources/X_w2v_test.pkl", "rb") as f:
    X_test = pickle.load(f)
with open("../resources/y_train.pkl", "rb") as f:
    y_train = pickle.load(f)
with open("../resources/y_test.pkl", "rb") as f:
    y_test = pickle.load(f)

In [3]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [4]:
svm_classifier = SVC(
    random_state=42
)

In [5]:
svm_classifier.fit(X_train,y_train)

SVC(random_state=42)

In [6]:

print(svm_classifier)

SVC(random_state=42)


In [9]:
# Dự đoán trên tập kiểm tra
y_pred_train = svm_classifier.predict(X_train)

# Đánh giá mô hình
accuracy_train = accuracy_score(y_train, y_pred_train)
report_train = classification_report(y_train, y_pred_train)

print(f'Accuracy: {accuracy_train}')
print('Classification Report:')
print(report_train)

Accuracy: 0.896
Classification Report:
              precision    recall  f1-score   support

    negative       0.90      0.89      0.90     17589
    positive       0.89      0.90      0.90     17411

    accuracy                           0.90     35000
   macro avg       0.90      0.90      0.90     35000
weighted avg       0.90      0.90      0.90     35000



In [10]:
# Dự đoán trên tập kiểm tra
y_pred = svm_classifier.predict(X_test)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.8911333333333333
Classification Report:
              precision    recall  f1-score   support

    negative       0.89      0.89      0.89      7411
    positive       0.89      0.89      0.89      7589

    accuracy                           0.89     15000
   macro avg       0.89      0.89      0.89     15000
weighted avg       0.89      0.89      0.89     15000



### Sử dụng tập bé để tìm ra các tham số thích hợp

In [4]:
# Lọc các dòng có nhãn 1 và -1
df_pos = df[df['sentiment'] == 'positive']
df_neg = df[df['sentiment'] == 'negative']

# Lấy ngẫu nhiên 2500 dòng có nhãn positive và 2300 dòng có nhãn negative
df_pos_sampled = df_pos.sample(n=2500, random_state=42)
df_neg_sampled = df_neg.sample(n=2300, random_state=42)

# Kết hợp lại thành tập dữ liệu mẫu
df_sampled = pd.concat([df_pos_sampled, df_neg_sampled])

# Kiểm tra kết quả
print(df_sampled['sentiment'].value_counts())

sentiment
positive    2500
negative    2300
Name: count, dtype: int64


In [30]:
#Xử lý dữ liệu như trên
df_sampled['review'] = df_sampled['review'].str.replace("<br />", "")
df_sampled['review'] = df_sampled['review'].str.translate(str.maketrans('', '', string.punctuation))
df_sampled['review'] = df_sampled['review'].str.strip()
df_sampled['review'] = df_sampled['review'].str.replace(r'\s+',' ', regex=True)
# Tải stop words từ nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def remove_stop_words(text):
    words = text.split()
    filter_words = [word for word in words if word not in stop_words]
    return ' '.join(filter_words)
df_sampled['review'] = df_sampled['review'].apply(remove_stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
#Chuyển nhã sang số
def sentiment_encode(text):
    if(text == "positive"):
        return 1
    return -1
df_sampled['sentiment'] = df_sampled['sentiment'].apply(sentiment_encode)

In [32]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(df_sampled['review'], df_sampled['sentiment'], test_size=0.3, random_state=42)


In [41]:
# Chuyển đổi văn bản thành các đặc trưng số sử dụng TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=500,)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [42]:
#SVM
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C': [10, 20, 50, 100],  # Regularization parameter
    'gamma': [0.001, 0.01, 0.1, 1.0],  # Kernel coefficient
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],  # Kernel type
    'degree': [3, 5],  # Chỉ có hiệu lực với kernel 'poly' (bậc đa thức)
    'coef0': [0, 0.1, 0.5],  # Chỉ có hiệu lực với kernel 'poly' và 'sigmoid'
}

grid_search = GridSearchCV(
    SVC(random_state=42),  # Mô hình SVM
    param_grid,             # Tham số tìm kiếm
)

In [43]:
print("X_train_tfidf shape:", X_train_tfidf.shape)
print("y_train shape:", y_train.shape)

X_train_tfidf shape: (3360, 500)
y_train shape: (3360,)


In [44]:
print("Unique classes in y_train:", set(y_train))
print("Class distribution:\n", pd.Series(y_train).value_counts())

Unique classes in y_train: {1, -1}
Class distribution:
 sentiment
 1    1733
-1    1627
Name: count, dtype: int64


In [45]:
grid_search.fit(X_train_tfidf, y_train)


GridSearchCV(estimator=SVC(random_state=42),
             param_grid={'C': [10, 20, 50, 100], 'coef0': [0, 0.1, 0.5],
                         'degree': [3, 5], 'gamma': [0.001, 0.01, 0.1, 1.0],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']})

In [46]:
best_model = grid_search.best_estimator_
# In ra mô hình tốt nhất và tham số tốt nhất
print("Mô hình tốt nhất:", best_model)
print("Các tham số tốt nhất:", grid_search.best_params_)

Mô hình tốt nhất: SVC(C=50, coef0=0.5, gamma=0.01, kernel='poly', random_state=42)
Các tham số tốt nhất: {'C': 50, 'coef0': 0.5, 'degree': 3, 'gamma': 0.01, 'kernel': 'poly'}


In [48]:
# Dự đoán trên tập kiểm tra
y_pred_train = grid_search.predict(X_train_tfidf)

# Đánh giá mô hình
accuracy_train = accuracy_score(y_train, y_pred_train)
report_train = classification_report(y_train, y_pred_train)

print(f'Accuracy: {accuracy_train}')
print('Classification Report:')
print(report_train)

Accuracy: 0.8675595238095238
Classification Report:
              precision    recall  f1-score   support

          -1       0.88      0.84      0.86      1627
           1       0.86      0.89      0.87      1733

    accuracy                           0.87      3360
   macro avg       0.87      0.87      0.87      3360
weighted avg       0.87      0.87      0.87      3360



In [49]:
# Dự đoán trên tập kiểm tra
y_pred = grid_search.predict(X_test_tfidf)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.8097222222222222
Classification Report:
              precision    recall  f1-score   support

          -1       0.81      0.78      0.79       673
           1       0.81      0.84      0.82       767

    accuracy                           0.81      1440
   macro avg       0.81      0.81      0.81      1440
weighted avg       0.81      0.81      0.81      1440

